In [2]:
import pandas as pd
import numpy as np

# Configuración
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)            # Ancho de línea amplio
pd.set_option('display.max_colwidth', 30)       # Limitar ancho de cada columna
pd.set_option('display.precision', 2)           # Precisión decimal

# Cargar datos limpios
banco_limpio = pd.read_csv('banco_limpio.csv')
cliente_limpio = pd.read_csv('cliente_limpio.csv')

# Reconvertir fecha (se pierde al guardar en CSV)
banco_limpio['date'] = pd.to_datetime(banco_limpio['date'])
print(banco_limpio.head())
print(cliente_limpio.head())

   age        job  marital    education  default  housing  loan    contact  duration  campaign  pdays  previous     poutcome  emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  nr.employed   y       date  latitude  longitude                            id_   año  mes  trimestre  dia_semana
0   38  housemaid  MARRIED     basic.4y      0.0      0.0   0.0  telephone       261         1    999         0  NONEXISTENT           1.1           93.99          -36.4       4.86       5191.0  no 2019-08-02     41.49     -71.23  089b39d8-e4d0-461b-87d4-81...  2019    8          3           4
1   57   services  MARRIED  high.school      0.0      0.0   0.0  telephone       149         1    999         0  NONEXISTENT           1.1           93.99          -36.4       4.86       5191.0  no 2016-09-14     34.60     -83.92  e9d37224-cb6f-4942-98d7-46...  2016    9          3           2
2   37   services  MARRIED  high.school      0.0      1.0   0.0  telephone       226         1    999         0  NO

In [3]:
## ANÁLISIS DESCRIPTIVO

# Lo básico que necesitas
print("Tamaño del dataset:", banco_limpio.shape)
print("\nTipos de datos:")
print(banco_limpio.dtypes)

print("\nEstadísticas básicas:")
print(banco_limpio.describe())

print("\nValores únicos de la variable objetivo:")
print(banco_limpio['y'].value_counts())

Tamaño del dataset: (43000, 27)

Tipos de datos:
age                        int64
job                       object
marital                   object
education                 object
default                  float64
housing                  float64
loan                     float64
contact                   object
duration                   int64
campaign                   int64
pdays                      int64
previous                   int64
poutcome                  object
emp.var.rate             float64
cons.price.idx           float64
cons.conf.idx            float64
euribor3m                float64
nr.employed              float64
y                         object
date              datetime64[ns]
latitude                 float64
longitude                float64
id_                       object
año                        int64
mes                        int64
trimestre                  int64
dia_semana                 int64
dtype: object

Estadísticas básicas:
            age   defau

In [4]:
# PASO 1: INFORMACIÓN GENERAL 
print("PASO 1: Información general")
print("-" * 30)
print(f"Tamaño del dataset: {banco_limpio.shape[0]:,} filas × {banco_limpio.shape[1]} columnas")
print(f"Periodo de datos: {banco_limpio['date'].min().strftime('%Y-%m-%d')} a {banco_limpio['date'].max().strftime('%Y-%m-%d')}")
print(f"Valores faltantes: {banco_limpio.isnull().sum().sum()} (dataset completo)")

PASO 1: Información general
------------------------------
Tamaño del dataset: 43,000 filas × 27 columnas
Periodo de datos: 2015-01-01 a 2019-12-31
Valores faltantes: 0 (dataset completo)


In [5]:
## PASO 2: ANÁLISIS DE LA VARIABLE OBJETIVO
print(f"\nPASO 2: Variable objetivo 'y' (¿suscribió el producto?)")
print("-" * 50)

conteos_y = banco_limpio['y'].value_counts()
porcentajes_y = banco_limpio['y'].value_counts(normalize=True) * 100

print("Distribución absoluta:")
for categoria, cantidad in conteos_y.items():
    porcentaje = porcentajes_y[categoria]
    print(f"  {categoria}: {cantidad:,} clientes ({porcentaje:.1f}%)")

print(f"\nConclusión: Dataset desbalanceado - solo {porcentajes_y['yes']:.1f}% acepta el producto")



PASO 2: Variable objetivo 'y' (¿suscribió el producto?)
--------------------------------------------------
Distribución absoluta:
  no: 38,156 clientes (88.7%)
  yes: 4,844 clientes (11.3%)

Conclusión: Dataset desbalanceado - solo 11.3% acepta el producto


In [6]:
# PASO 3: ANÁLISIS DE VARIABLES 

# Edad de los clientes

print(f"  Edad promedio: {banco_limpio['age'].mean():.1f} años")
print(f"  Mediana: {banco_limpio['age'].median():.0f} años")
print(f"  Rango: {banco_limpio['age'].min()}-{banco_limpio['age'].max()} años")
print(f"  Desviación estándar: {banco_limpio['age'].std():.1f} años")


  Edad promedio: 39.7 años
  Mediana: 38 años
  Rango: 17-98 años
  Desviación estándar: 9.8 años


In [7]:

# Crear grupos de edad para análisis
banco_limpio['grupo_edad'] = pd.cut(banco_limpio['age'], 
                                   bins=[0, 30, 40, 50, 60, 100],
                                   labels=['18-30', '31-40', '41-50', '51-60', '60+'])

print("\n  Distribución por grupos de edad:")
distribucion_edad = banco_limpio['grupo_edad'].value_counts().sort_index()
for grupo, cantidad in distribucion_edad.items():
    porcentaje = (cantidad / len(banco_limpio)) * 100
    print(f"    {grupo}: {cantidad:,} ({porcentaje:.1f}%)")


  Distribución por grupos de edad:
    18-30: 6,871 (16.0%)
    31-40: 20,191 (47.0%)
    41-50: 9,378 (21.8%)
    51-60: 5,714 (13.3%)
    60+: 846 (2.0%)


In [8]:
# Estado civil
print("\n3.3 ESTADO CIVIL:")
estado_civil = banco_limpio['marital'].value_counts()
for estado, cantidad in estado_civil.items():
    porcentaje = (cantidad / len(banco_limpio)) * 100
    print(f"    {estado}: {cantidad:,} ({porcentaje:.1f}%)")


3.3 ESTADO CIVIL:
    MARRIED: 26,084 (60.7%)
    SINGLE: 12,105 (28.2%)
    DIVORCED: 4,811 (11.2%)


In [9]:
# Educación
print("\n3.4 EDUCACIÓN:")
educacion = banco_limpio['education'].value_counts()
for nivel, cantidad in educacion.items():
    porcentaje = (cantidad / len(banco_limpio)) * 100
    print(f"    {nivel}: {cantidad:,} ({porcentaje:.1f}%)")



3.4 EDUCACIÓN:
    university.degree: 14,529 (33.8%)
    high.school: 9,925 (23.1%)
    basic.9y: 6,309 (14.7%)
    professional.course: 5,477 (12.7%)
    basic.4y: 4,356 (10.1%)
    basic.6y: 2,386 (5.5%)
    illiterate: 18 (0.0%)


In [10]:
#4.INSTRUMENTOS FINANCIEROS
print("4.1 PRÉSTAMOS:")
print(f"  Tienen hipoteca: {banco_limpio['housing'].sum():,} ({(banco_limpio['housing'].mean()*100):.1f}%)")
print(f"  Tienen otro préstamo: {banco_limpio['loan'].sum():,} ({(banco_limpio['loan'].mean()*100):.1f}%)")
print(f"  Sin préstamos: {((banco_limpio['housing'] == 0) & (banco_limpio['loan'] == 0)).sum():,}")

4.1 PRÉSTAMOS:
  Tienen hipoteca: 22,498.0 (52.3%)
  Tienen otro préstamo: 6,532.0 (15.2%)
  Sin préstamos: 17,816


In [11]:
print(f"\n4.2 HISTORIAL CREDITICIO:")
defaults = banco_limpio['default'].sum()
print(f"  Con historial de impago: {defaults:,} ({(defaults/len(banco_limpio)*100):.3f}%)")
print("  Interpretación: Muy pocos clientes con problemas crediticios")


4.2 HISTORIAL CREDITICIO:
  Con historial de impago: 3.0 (0.007%)
  Interpretación: Muy pocos clientes con problemas crediticios


In [12]:
#5.Duración llamadas
print("5.1 DURACIÓN DE LLAMADAS:")
print(f"  Promedio: {banco_limpio['duration'].mean():.0f} segundos ({banco_limpio['duration'].mean()/60:.1f} minutos)")
print(f"  Mediana: {banco_limpio['duration'].median():.0f} segundos ")
print(f"  Llamada más larga: {banco_limpio['duration'].max():,} segundos ({banco_limpio['duration'].max()/60:.1f} minutos)")


5.1 DURACIÓN DE LLAMADAS:
  Promedio: 258 segundos (4.3 minutos)
  Mediana: 179 segundos 
  Llamada más larga: 4,918 segundos (82.0 minutos)


In [13]:
# Categorizar duración
banco_limpio['duracion_categoria'] = pd.cut(banco_limpio['duration']/60, 
                                           bins=[0, 2, 5, 10, float('inf')],
                                           labels=['Muy corta (<2min)', 'Corta (2-5min)', 
                                                  'Media (5-10min)', 'Larga (>10min)'])
print("  Distribución por duración:")
duracion_dist = banco_limpio['duracion_categoria'].value_counts()
for categoria, cantidad in duracion_dist.items():
    porcentaje = (cantidad / len(banco_limpio)) * 100
    print(f"    {categoria}: {cantidad:,} ({porcentaje:.1f}%)")                                                  

  Distribución por duración:
    Corta (2-5min): 17,842 (41.5%)
    Muy corta (<2min): 13,494 (31.4%)
    Media (5-10min): 8,066 (18.8%)
    Larga (>10min): 3,594 (8.4%)


In [14]:
print(f"\n5.2 NÚMERO DE CONTACTOS:")
print(f"  Promedio contactos por cliente: {banco_limpio['campaign'].mean():.1f}")
print(f"  Máximo contactos: {banco_limpio['campaign'].max()}")

contactos_dist = banco_limpio['campaign'].value_counts().head()
print("  Distribución de contactos:")
for contactos, cantidad in contactos_dist.items():
    porcentaje = (cantidad / len(banco_limpio)) * 100
    print(f"    {contactos} contacto(s): {cantidad:,} ({porcentaje:.1f}%)")




5.2 NÚMERO DE CONTACTOS:
  Promedio contactos por cliente: 2.6
  Máximo contactos: 56
  Distribución de contactos:
    1 contacto(s): 18,404 (42.8%)
    2 contacto(s): 11,048 (25.7%)
    3 contacto(s): 5,584 (13.0%)
    4 contacto(s): 2,777 (6.5%)
    5 contacto(s): 1,658 (3.9%)


In [15]:
# PASO 6: CREAR NUEVAS COLUMNAS 
print("Crear nuevas columnas")
# Columna simple: convertir yes/no a números
banco_limpio['exito'] = banco_limpio['y'].map({'yes': 1, 'no': 0})
print("1.1 Nueva columna 'exito': yes=1, no=0")
print(banco_limpio['exito'].head())

Crear nuevas columnas
1.1 Nueva columna 'exito': yes=1, no=0
0    0
1    0
2    0
3    0
4    0
Name: exito, dtype: int64


In [16]:
# Columna con cálculo: duración en minutos
banco_limpio['duracion_min'] = banco_limpio['duration'] / 60
print("\n1.2 Nueva columna 'duracion_min': segundos dividido por 60")
print(banco_limpio['duracion_min'].head())



1.2 Nueva columna 'duracion_min': segundos dividido por 60
0    4.35
1    2.48
2    3.77
3    2.52
4    5.12
Name: duracion_min, dtype: float64


In [17]:
# Columna con categorías simples
banco_limpio['edad_grupo'] = 'adulto'  # Valor por defecto
banco_limpio.loc[banco_limpio['age'] < 30, 'edad_grupo'] = 'joven'
banco_limpio.loc[banco_limpio['age'] >= 60, 'edad_grupo'] = 'mayor'
print("\n1.3 Nueva columna 'edad_grupo': joven, adulto, mayor")
print(banco_limpio['edad_grupo'].value_counts())

print(f"\nAhora tenemos {banco_limpio.shape[1]} columnas (antes teníamos menos)")



1.3 Nueva columna 'edad_grupo': joven, adulto, mayor
edad_grupo
adulto    36637
joven      5266
mayor      1097
Name: count, dtype: int64

Ahora tenemos 32 columnas (antes teníamos menos)


In [18]:
# paso 7: promedio por grupo
print("Edad promedio por grupo de edad:")
promedio_por_grupo = banco_limpio.groupby('edad_grupo')['age'].mean()
print(promedio_por_grupo)

Edad promedio por grupo de edad:
edad_grupo
adulto    40.78
joven     26.52
mayor     68.41
Name: age, dtype: float64


In [44]:
# GroupBy con conteos
print("\n Cuántas personas hay en cada trabajo:")
conteo_trabajo = banco_limpio.groupby('job').size()
print(conteo_trabajo.head())


 Cuántas personas hay en cada trabajo:
job
admin.          11218
blue-collar      9654
entrepreneur     1522
housemaid        1123
management       3050
dtype: int64


In [19]:
# GroupBy con tasa de éxito
print("\n2.3 Tasa de éxito por trabajo (muy importante):")
tasa_por_trabajo = banco_limpio.groupby('job')['exito'].mean()
print("Top 5 trabajos con mejor tasa:")
print(tasa_por_trabajo.sort_values(ascending=False).head())



2.3 Tasa de éxito por trabajo (muy importante):
Top 5 trabajos con mejor tasa:
job
student       0.31
retired       0.25
unemployed    0.14
admin.        0.13
management    0.11
Name: exito, dtype: float64


In [20]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)            # Ancho de línea amplio
pd.set_option('display.max_colwidth', 30)       # Limitar ancho de cada columna
pd.set_option('display.precision', 2)           # Precisión decimal
print(banco_limpio.head())

   age        job  marital    education  default  housing  loan    contact  duration  campaign  pdays  previous     poutcome  emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  nr.employed   y       date  latitude  longitude                            id_   año  mes  trimestre  dia_semana grupo_edad duracion_categoria  exito  duracion_min edad_grupo
0   38  housemaid  MARRIED     basic.4y      0.0      0.0   0.0  telephone       261         1    999         0  NONEXISTENT           1.1           93.99          -36.4       4.86       5191.0  no 2019-08-02     41.49     -71.23  089b39d8-e4d0-461b-87d4-81...  2019    8          3           4      31-40     Corta (2-5min)      0          4.35     adulto
1   57   services  MARRIED  high.school      0.0      0.0   0.0  telephone       149         1    999         0  NONEXISTENT           1.1           93.99          -36.4       4.86       5191.0  no 2016-09-14     34.60     -83.92  e9d37224-cb6f-4942-98d7-46...  2016    9          3    

In [21]:
# tiempo promedio de clientes que aceptaron el producto
tiempo_promedio_exito = banco_limpio[banco_limpio['exito'] == 1]['duration'].mean()
print(f"\nClientes que aceptaron el producto tardaron en promedio {tiempo_promedio_exito/60:.1f} minutos en la llamada")


Clientes que aceptaron el producto tardaron en promedio 9.2 minutos en la llamada


In [22]:
# paso8 apply: función que cuenta cosas por grupo de edad
pd.set_option('display.max_colwidth', None)

def contar_info_grupo(grupo):
    total = len(grupo)
    exitos = grupo['exito'].sum()
    return f"Total: {total}, Éxitos: {exitos}, Porcentaje: {(exitos/total*100):.1f}%"

print("3.1 Información por grupo de edad:")
info_grupos = banco_limpio.groupby('edad_grupo').apply(contar_info_grupo)
print(info_grupos)

3.1 Información por grupo de edad:
edad_grupo
adulto    Total: 36637, Éxitos: 3563, Porcentaje: 9.7%
joven      Total: 5266, Éxitos: 849, Porcentaje: 16.1%
mayor      Total: 1097, Éxitos: 432, Porcentaje: 39.4%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\2162633029.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = banco_limpio.groupby('edad_grupo').apply(contar_info_grupo)


In [23]:
# paso8 apply: función que cuenta cosas por trabajo
print("Información por grupo de trabajo:")
info_grupos = banco_limpio.groupby('job').apply(contar_info_grupo)
print(info_grupos)

Información por grupo de trabajo:
job
admin.           Total: 11218, Éxitos: 1454, Porcentaje: 13.0%
blue-collar         Total: 9654, Éxitos: 665, Porcentaje: 6.9%
entrepreneur        Total: 1522, Éxitos: 126, Porcentaje: 8.3%
housemaid           Total: 1123, Éxitos: 111, Porcentaje: 9.9%
management         Total: 3050, Éxitos: 342, Porcentaje: 11.2%
retired            Total: 1790, Éxitos: 451, Porcentaje: 25.2%
self-employed      Total: 1489, Éxitos: 161, Porcentaje: 10.8%
services            Total: 4162, Éxitos: 336, Porcentaje: 8.1%
student             Total: 903, Éxitos: 283, Porcentaje: 31.3%
technician         Total: 7026, Éxitos: 762, Porcentaje: 10.8%
unemployed         Total: 1063, Éxitos: 153, Porcentaje: 14.4%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\1512695645.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = banco_limpio.groupby('job').apply(contar_info_grupo)


In [24]:
# paso8 apply: función que cuenta cosas por educación
print("Información por grupo de educación:")
info_grupos = banco_limpio.groupby('education').apply(contar_info_grupo)
print(info_grupos)

Información por grupo de educación:
education
basic.4y                 Total: 4356, Éxitos: 448, Porcentaje: 10.3%
basic.6y                  Total: 2386, Éxitos: 194, Porcentaje: 8.1%
basic.9y                  Total: 6309, Éxitos: 493, Porcentaje: 7.8%
high.school             Total: 9925, Éxitos: 1076, Porcentaje: 10.8%
illiterate                   Total: 18, Éxitos: 4, Porcentaje: 22.2%
professional.course      Total: 5477, Éxitos: 620, Porcentaje: 11.3%
university.degree      Total: 14529, Éxitos: 2009, Porcentaje: 13.8%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\3427374620.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = banco_limpio.groupby('education').apply(contar_info_grupo)


In [25]:
# paso8 apply: función que cuenta cosas por estado civil
print("Información por grupo de estado civil:")
info_grupos = banco_limpio.groupby('marital').apply(contar_info_grupo)
print(info_grupos)

Información por grupo de estado civil:
marital
DIVORCED      Total: 4811, Éxitos: 490, Porcentaje: 10.2%
MARRIED     Total: 26084, Éxitos: 2668, Porcentaje: 10.2%
SINGLE      Total: 12105, Éxitos: 1686, Porcentaje: 13.9%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\2916522204.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = banco_limpio.groupby('marital').apply(contar_info_grupo)


In [26]:
# paso8 apply: función que cuenta cosas por hipoteca
print("Información por grupo de hipoteca:")
info_grupos = banco_limpio.groupby('housing').apply(contar_info_grupo)
print(info_grupos)

Información por grupo de hipoteca:
housing
0.0    Total: 20502, Éxitos: 2228, Porcentaje: 10.9%
1.0    Total: 22498, Éxitos: 2616, Porcentaje: 11.6%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\1575003062.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = banco_limpio.groupby('housing').apply(contar_info_grupo)


In [27]:
# paso8 apply: función que cuenta cosas por prestamo
print("Información por grupo de prestamo:")
info_grupos = banco_limpio.groupby('loan').apply(contar_info_grupo)
print(info_grupos)

Información por grupo de prestamo:
loan
0.0    Total: 36468, Éxitos: 4131, Porcentaje: 11.3%
1.0      Total: 6532, Éxitos: 713, Porcentaje: 10.9%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\2724272654.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = banco_limpio.groupby('loan').apply(contar_info_grupo)


In [28]:
# paso8 apply: función que cuenta cosas por resultado campaña anterior
print("Información por campaña anterior:")
info_grupos = banco_limpio.groupby('poutcome').apply(contar_info_grupo)
print(info_grupos)

Información por campaña anterior:
poutcome
FAILURE         Total: 4461, Éxitos: 635, Porcentaje: 14.2%
NONEXISTENT    Total: 37103, Éxitos: 3271, Porcentaje: 8.8%
SUCCESS         Total: 1436, Éxitos: 938, Porcentaje: 65.3%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\1074517590.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = banco_limpio.groupby('poutcome').apply(contar_info_grupo)


In [29]:
# paso8 apply: función que cuenta cosas por año
print("Información por año:")
info_grupos = banco_limpio.groupby('año').apply(contar_info_grupo)
print(info_grupos)

Información por año:
año
2015    Total: 8544, Éxitos: 929, Porcentaje: 10.9%
2016    Total: 8533, Éxitos: 997, Porcentaje: 11.7%
2017    Total: 8810, Éxitos: 972, Porcentaje: 11.0%
2018    Total: 8549, Éxitos: 979, Porcentaje: 11.5%
2019    Total: 8564, Éxitos: 967, Porcentaje: 11.3%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\4133958330.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = banco_limpio.groupby('año').apply(contar_info_grupo)


In [30]:
# paso8 apply: función que cuenta cosas por mes
print("Información por mes:")
info_grupos = banco_limpio.groupby('mes').apply(contar_info_grupo)
print(info_grupos)

Información por mes:
mes
1     Total: 3517, Éxitos: 408, Porcentaje: 11.6%
2     Total: 3577, Éxitos: 398, Porcentaje: 11.1%
3     Total: 3589, Éxitos: 385, Porcentaje: 10.7%
4     Total: 3582, Éxitos: 401, Porcentaje: 11.2%
5     Total: 3547, Éxitos: 377, Porcentaje: 10.6%
6     Total: 3538, Éxitos: 401, Porcentaje: 11.3%
7     Total: 3841, Éxitos: 443, Porcentaje: 11.5%
8     Total: 3526, Éxitos: 404, Porcentaje: 11.5%
9     Total: 3569, Éxitos: 366, Porcentaje: 10.3%
10    Total: 3599, Éxitos: 445, Porcentaje: 12.4%
11    Total: 3603, Éxitos: 414, Porcentaje: 11.5%
12    Total: 3512, Éxitos: 402, Porcentaje: 11.4%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\2031704570.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = banco_limpio.groupby('mes').apply(contar_info_grupo)


In [31]:
# paso8 apply: función que cuenta cosas por dia de la semana
print("Información por dia de la semana:")
info_grupos = banco_limpio.groupby('dia_semana').apply(contar_info_grupo)
print(info_grupos)

Información por dia de la semana:
dia_semana
0    Total: 6035, Éxitos: 657, Porcentaje: 10.9%
1    Total: 6032, Éxitos: 686, Porcentaje: 11.4%
2    Total: 5979, Éxitos: 656, Porcentaje: 11.0%
3    Total: 6198, Éxitos: 748, Porcentaje: 12.1%
4    Total: 6212, Éxitos: 673, Porcentaje: 10.8%
5    Total: 6395, Éxitos: 732, Porcentaje: 11.4%
6    Total: 6149, Éxitos: 692, Porcentaje: 11.3%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\711535200.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = banco_limpio.groupby('dia_semana').apply(contar_info_grupo)


In [32]:
# calcula metricas por grupo
def calcular_metricas(grupo):
        return pd.Series({
        'total_personas': len(grupo),
        'tasa_exito': grupo['exito'].mean(),
        'edad_promedio': grupo['age'].mean()
    })

print("\nMétricas por tipo de trabajo (top 5):")
metricas_trabajo = banco_limpio.groupby('job').apply(calcular_metricas)
print(metricas_trabajo.sort_values('tasa_exito', ascending=False).head())


Métricas por tipo de trabajo (top 5):
            total_personas  tasa_exito  edad_promedio
job                                                  
student              903.0        0.31          27.09
retired             1790.0        0.25          59.30
unemployed          1063.0        0.14          39.40
admin.             11218.0        0.13          38.22
management          3050.0        0.11          41.96


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\2596457538.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metricas_trabajo = banco_limpio.groupby('job').apply(calcular_metricas)


In [33]:
#Crear tablas con el mismo índice (trabajo)
tabla_financiera = banco_limpio.groupby('job')[['housing', 'loan']].mean()
tabla_edad = banco_limpio.groupby('job')['age'].mean().to_frame('edad_prom')

print("Tabla financiera:")
print(tabla_financiera.head())

print("\nTabla de edad:")
print(tabla_edad.head())

# 6.2 Juntarlas con join
tabla_unida = tabla_financiera.join(tabla_edad)
print("\n6.2 Tablas unidas con join:")
print(tabla_unida.head())

Tabla financiera:
              housing  loan
job                        
admin.           0.53  0.16
blue-collar      0.51  0.15
entrepreneur     0.53  0.14
housemaid        0.51  0.14
management       0.51  0.15

Tabla de edad:
              edad_prom
job                    
admin.            38.22
blue-collar       39.36
entrepreneur      41.33
housemaid         44.45
management        41.96

6.2 Tablas unidas con join:
              housing  loan  edad_prom
job                                   
admin.           0.53  0.16      38.22
blue-collar      0.51  0.15      39.36
entrepreneur     0.53  0.14      41.33
housemaid        0.51  0.14      44.45
management       0.51  0.15      41.96


In [34]:
# PASO 9: PIVOT (tabla dinámica)
print(f"Pivot - Tabla dinámica")
print("-"*30)

# Crear tabla pivot 
print("Tabla pivot: Tasa de éxito por edad y educación")
tabla_pivot = banco_limpio.pivot_table(
    values='exito',
    index='edad_grupo',
    columns='education',
    aggfunc='mean'
).round(3)
print(tabla_pivot)

Pivot - Tabla dinámica
------------------------------
Tabla pivot: Tasa de éxito por edad y educación
education   basic.4y  basic.6y  basic.9y  high.school  illiterate  professional.course  university.degree
edad_grupo                                                                                               
adulto          0.07      0.08      0.07         0.09        0.19                 0.10               0.12
joven           0.09      0.11      0.12         0.16         NaN                 0.16               0.20
mayor           0.45      0.26      0.19         0.39        0.50                 0.37               0.39


In [35]:
# PASO 10: MELT (reducir columnas)
print(f"\nMelt - Convertir a formato corto")
print("-"*38)

# Crear datos en una columna
datos_conjunto = banco_limpio.groupby('job')[['housing', 'loan', 'default']].mean().reset_index()
print("Datos en formato conjunto:")
print(datos_conjunto.head())

# Convertir a formato corto con melt
datos_corto = datos_conjunto.melt(
    id_vars='job',
    value_vars=['housing', 'loan', 'default'],
    var_name='tipo_prestamo',
    value_name='porcentaje'
)
print(f"\n Datos en formato corto (primeras 10 filas):")
print(datos_corto.head(10))


Melt - Convertir a formato corto
--------------------------------------
Datos en formato conjunto:
            job  housing  loan  default
0        admin.     0.53  0.16      0.0
1   blue-collar     0.51  0.15      0.0
2  entrepreneur     0.53  0.14      0.0
3     housemaid     0.51  0.14      0.0
4    management     0.51  0.15      0.0

 Datos en formato corto (primeras 10 filas):
             job tipo_prestamo  porcentaje
0         admin.       housing        0.53
1    blue-collar       housing        0.51
2   entrepreneur       housing        0.53
3      housemaid       housing        0.51
4     management       housing        0.51
5        retired       housing        0.52
6  self-employed       housing        0.52
7       services       housing        0.52
8        student       housing        0.53
9     technician       housing        0.54


In [36]:
banco_limpio.head()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,date,latitude,longitude,id_,año,mes,trimestre,dia_semana,grupo_edad,duracion_categoria,exito,duracion_min,edad_grupo
0,38,housemaid,MARRIED,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,NONEXISTENT,1.1,93.99,-36.4,4.86,5191.0,no,2019-08-02,41.49,-71.23,089b39d8-e4d0-461b-87d4-814d71e0e079,2019,8,3,4,31-40,Corta (2-5min),0,4.35,adulto
1,57,services,MARRIED,high.school,0.0,0.0,0.0,telephone,149,1,999,0,NONEXISTENT,1.1,93.99,-36.4,4.86,5191.0,no,2016-09-14,34.60,-83.92,e9d37224-cb6f-4942-98d7-46672963d097,2016,9,3,2,51-60,Corta (2-5min),0,2.48,adulto
2,37,services,MARRIED,high.school,0.0,1.0,0.0,telephone,226,1,999,0,NONEXISTENT,1.1,93.99,-36.4,4.86,5191.0,no,2019-02-15,34.94,-94.85,3f9f49b5-e410-4948-bf6e-f9244f04918b,2019,2,1,4,31-40,Corta (2-5min),0,3.77,adulto
3,40,admin.,MARRIED,basic.6y,0.0,0.0,0.0,telephone,151,1,999,0,NONEXISTENT,1.1,93.99,-36.4,4.86,5191.0,no,2015-11-29,49.04,-70.31,9991fafb-4447-451a-8be2-b0df6098d13e,2015,11,4,6,31-40,Corta (2-5min),0,2.52,adulto
4,56,services,MARRIED,high.school,0.0,0.0,1.0,telephone,307,1,999,0,NONEXISTENT,1.1,93.99,-36.4,4.86,5191.0,no,2017-01-29,38.03,-104.46,eca60b76-70b6-4077-80ba-bc52e8ebb0eb,2017,1,1,6,51-60,Media (5-10min),0,5.12,adulto


In [37]:
# IDs de banco_limpio
print("Primeros 5 IDs de banco_limpio:")
print(banco_limpio['id_'].head().tolist())
print(banco_limpio.shape)

# IDs de cliente_limpio  
print("\nPrimeros 5 IDs de cliente_limpio:")
print(cliente_limpio['ID'].head().tolist())
print(cliente_limpio.shape)

Primeros 5 IDs de banco_limpio:
['089b39d8-e4d0-461b-87d4-814d71e0e079', 'e9d37224-cb6f-4942-98d7-46672963d097', '3f9f49b5-e410-4948-bf6e-f9244f04918b', '9991fafb-4447-451a-8be2-b0df6098d13e', 'eca60b76-70b6-4077-80ba-bc52e8ebb0eb']
(43000, 32)

Primeros 5 IDs de cliente_limpio:
['089b39d8-e4d0-461b-87d4-814d71e0e079', 'e9d37224-cb6f-4942-98d7-46672963d097', '3f9f49b5-e410-4948-bf6e-f9244f04918b', '9991fafb-4447-451a-8be2-b0df6098d13e', 'eca60b76-70b6-4077-80ba-bc52e8ebb0eb']
(20115, 6)


In [38]:
print(cliente_limpio.dtypes)

Income                int64
Kidhome               int64
Teenhome              int64
Dt_Customer          object
NumWebVisitsMonth     int64
ID                   object
dtype: object


In [39]:
# Renombrar columnas para que coincidan si es necesario
if 'id_' in banco_limpio.columns and 'ID' in cliente_limpio.columns:
    # Crear una copia del cliente con nombre consistente
    cliente_rename = cliente_limpio.rename(columns={'ID': 'id_'})

In [40]:
# INNER: Solo registros que están en ambos datasets
merge_inner = banco_limpio.merge(cliente_rename, on='id_', how='inner')
print(f"INNER (coincidencias): {len(merge_inner)} registros")

INNER (coincidencias): 20018 registros


In [41]:
merge_inner.head()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,date,latitude,longitude,id_,año,mes,trimestre,dia_semana,grupo_edad,duracion_categoria,exito,duracion_min,edad_grupo,Income,Kidhome,Teenhome,Dt_Customer,NumWebVisitsMonth
0,38,housemaid,MARRIED,basic.4y,0.0,0.0,0.0,telephone,261,1,999,0,NONEXISTENT,1.1,93.99,-36.4,4.86,5191.0,no,2019-08-02,41.49,-71.23,089b39d8-e4d0-461b-87d4-814d71e0e079,2019,8,3,4,31-40,Corta (2-5min),0,4.35,adulto,161770,1,0,2012-04-04,29
1,57,services,MARRIED,high.school,0.0,0.0,0.0,telephone,149,1,999,0,NONEXISTENT,1.1,93.99,-36.4,4.86,5191.0,no,2016-09-14,34.60,-83.92,e9d37224-cb6f-4942-98d7-46672963d097,2016,9,3,2,51-60,Corta (2-5min),0,2.48,adulto,85477,1,1,2012-12-30,7
2,37,services,MARRIED,high.school,0.0,1.0,0.0,telephone,226,1,999,0,NONEXISTENT,1.1,93.99,-36.4,4.86,5191.0,no,2019-02-15,34.94,-94.85,3f9f49b5-e410-4948-bf6e-f9244f04918b,2019,2,1,4,31-40,Corta (2-5min),0,3.77,adulto,147233,1,1,2012-02-02,5
3,40,admin.,MARRIED,basic.6y,0.0,0.0,0.0,telephone,151,1,999,0,NONEXISTENT,1.1,93.99,-36.4,4.86,5191.0,no,2015-11-29,49.04,-70.31,9991fafb-4447-451a-8be2-b0df6098d13e,2015,11,4,6,31-40,Corta (2-5min),0,2.52,adulto,121393,1,2,2012-12-21,29
4,56,services,MARRIED,high.school,0.0,0.0,1.0,telephone,307,1,999,0,NONEXISTENT,1.1,93.99,-36.4,4.86,5191.0,no,2017-01-29,38.03,-104.46,eca60b76-70b6-4077-80ba-bc52e8ebb0eb,2017,1,1,6,51-60,Media (5-10min),0,5.12,adulto,63164,1,2,2012-06-20,20


In [42]:

print(merge_inner.dtypes)

age                            int64
job                           object
marital                       object
education                     object
default                      float64
housing                      float64
loan                         float64
contact                       object
duration                       int64
campaign                       int64
pdays                          int64
previous                       int64
poutcome                      object
emp.var.rate                 float64
cons.price.idx               float64
cons.conf.idx                float64
euribor3m                    float64
nr.employed                  float64
y                             object
date                  datetime64[ns]
latitude                     float64
longitude                    float64
id_                           object
año                            int64
mes                            int64
trimestre                      int64
dia_semana                     int64
g

In [43]:
# Crear categorías de Income en merge_inner
merge_inner['categoria_ingresos'] = 'ingresos_medios'  # Valor por defecto
merge_inner.loc[merge_inner['Income'] < 50000, 'categoria_ingresos'] = 'ingresos_bajos'
merge_inner.loc[merge_inner['Income'] >= 80000, 'categoria_ingresos'] = 'ingresos_altos'

print("Distribución de categorías de ingresos:")
print(merge_inner['categoria_ingresos'].value_counts())

# Análisis de tasa de éxito por ingresos
print("\nTasa de éxito por categoría de ingresos:")
tasa_por_ingresos = merge_inner.groupby('categoria_ingresos')['exito'].mean()
print(tasa_por_ingresos)

# Ver estadísticas básicas de Income
print(f"\nEstadísticas de Income:")
print(f"Mínimo: ${merge_inner['Income'].min():,.0f}")
print(f"Mediana: ${merge_inner['Income'].median():,.0f}")
print(f"Máximo: ${merge_inner['Income'].max():,.0f}")
print(f"Promedio: ${merge_inner['Income'].mean():,.0f}")

Distribución de categorías de ingresos:
categoria_ingresos
ingresos_altos     11505
ingresos_bajos      5116
ingresos_medios     3397
Name: count, dtype: int64

Tasa de éxito por categoría de ingresos:
categoria_ingresos
ingresos_altos     0.05
ingresos_bajos     0.05
ingresos_medios    0.04
Name: exito, dtype: float64

Estadísticas de Income:
Mínimo: $5,852
Mediana: $92,974
Máximo: $180,791
Promedio: $93,072


In [44]:
# función que cuenta cosas por sueldo
print("Información por grupo de categoria_ingresos:")
info_grupos = merge_inner.groupby('categoria_ingresos').apply(contar_info_grupo)
print(info_grupos)

Información por grupo de categoria_ingresos:
categoria_ingresos
ingresos_altos     Total: 11505, Éxitos: 523, Porcentaje: 4.5%
ingresos_bajos      Total: 5116, Éxitos: 250, Porcentaje: 4.9%
ingresos_medios     Total: 3397, Éxitos: 145, Porcentaje: 4.3%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\822999435.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = merge_inner.groupby('categoria_ingresos').apply(contar_info_grupo)


In [45]:
# función que cuenta cosas por niños
print("Información por grupo de niños:")
info_grupos = merge_inner.groupby('Kidhome').apply(contar_info_grupo)
print(info_grupos)

Información por grupo de niños:
Kidhome
0    Total: 6629, Éxitos: 316, Porcentaje: 4.8%
1    Total: 6667, Éxitos: 294, Porcentaje: 4.4%
2    Total: 6722, Éxitos: 308, Porcentaje: 4.6%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\1352998264.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = merge_inner.groupby('Kidhome').apply(contar_info_grupo)


In [46]:
# función que cuenta cosas por adolescentes
print("Información por grupo de adolescentes:")
info_grupos = merge_inner.groupby('Teenhome').apply(contar_info_grupo)
print(info_grupos)

Información por grupo de adolescentes:
Teenhome
0    Total: 6659, Éxitos: 297, Porcentaje: 4.5%
1    Total: 6709, Éxitos: 324, Porcentaje: 4.8%
2    Total: 6650, Éxitos: 297, Porcentaje: 4.5%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\2372872426.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = merge_inner.groupby('Teenhome').apply(contar_info_grupo)


In [47]:
# Ver qué tipo es la columna
print("Tipo de Dt_Customer:", merge_inner['Dt_Customer'].dtype)
print("Primeros valores:", merge_inner['Dt_Customer'].head())

# Convertir a datetime primero
merge_inner['Dt_Customer'] = pd.to_datetime(merge_inner['Dt_Customer'])

# Ahora sí crear la columna de año
merge_inner['año_cliente'] = merge_inner['Dt_Customer'].dt.year

# Agrupar por año
print("Información por año del cliente:")
info_grupos = merge_inner.groupby('año_cliente').apply(contar_info_grupo)
print(info_grupos)

Tipo de Dt_Customer: object
Primeros valores: 0    2012-04-04
1    2012-12-30
2    2012-02-02
3    2012-12-21
4    2012-06-20
Name: Dt_Customer, dtype: object
Información por año del cliente:
año_cliente
2012    Total: 20018, Éxitos: 918, Porcentaje: 4.6%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\80412637.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = merge_inner.groupby('año_cliente').apply(contar_info_grupo)


In [48]:
# Crear categorías con umbrales más apropiados
print(f"Cuartiles de visitas:")
print(merge_inner['NumWebVisitsMonth'].describe())

# Usar cuartiles para las categorías
q33 = merge_inner['NumWebVisitsMonth'].quantile(0.33)
q66 = merge_inner['NumWebVisitsMonth'].quantile(0.66)

merge_inner['visitas_mensuales'] = 'visitas_medias'
merge_inner.loc[merge_inner['NumWebVisitsMonth'] <= q33, 'visitas_mensuales'] = 'pocas_visitas'
merge_inner.loc[merge_inner['NumWebVisitsMonth'] > q66, 'visitas_mensuales'] = 'muchas_visitas'

# Verificar distribución
print("\nNueva distribución:")
print(merge_inner['visitas_mensuales'].value_counts())

# Análisis por grupos
info_grupos = merge_inner.groupby('visitas_mensuales').apply(contar_info_grupo)
print(info_grupos)

Cuartiles de visitas:
count    20018.00
mean        16.54
std          9.23
min          1.00
25%          9.00
50%         16.00
75%         25.00
max         32.00
Name: NumWebVisitsMonth, dtype: float64

Nueva distribución:
visitas_mensuales
pocas_visitas     6872
visitas_medias    6859
muchas_visitas    6287
Name: count, dtype: int64
visitas_mensuales
muchas_visitas    Total: 6287, Éxitos: 273, Porcentaje: 4.3%
pocas_visitas     Total: 6872, Éxitos: 325, Porcentaje: 4.7%
visitas_medias    Total: 6859, Éxitos: 320, Porcentaje: 4.7%
dtype: object


C:\Users\USER\AppData\Local\Temp\ipykernel_2996\1931027752.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  info_grupos = merge_inner.groupby('visitas_mensuales').apply(contar_info_grupo)


In [49]:
# Guardar los datasets procesados
print("Guardando datasets para visualización...")

# Dataset principal limpio
banco_limpio.to_csv('banco_analizado.csv', index=False)
print(f"banco_analizado guardado: {banco_limpio.shape}")

# Dataset de cliente limpio
cliente_limpio.to_csv('cliente_analizado.csv', index=False)
print(f"cliente_analizado guardado: {cliente_limpio.shape}")

# Dataset combinado (merge)
merge_inner.to_csv('datos_combinados.csv', index=False)
print(f"datos_combinados guardado: {merge_inner.shape}")

print("\nArchivos guardados exitosamente en el directorio actual.")
print("Listos para cargar en el nuevo archivo de visualización")




Guardando datasets para visualización...
banco_analizado guardado: (43000, 32)
cliente_analizado guardado: (20115, 6)
datos_combinados guardado: (20018, 40)

Archivos guardados exitosamente en el directorio actual.
Listos para cargar en el nuevo archivo de visualización
